# 個体間関係のラベル付け

## おまじないパート

In [1]:
import sys
from pathlib import Path

# 現在のノートブックのパスを取得
notebook_dir = Path().resolve() 

# プロジェクトルートディレクトリ（notebooksディレクトリの親ディレクトリ）を取得
# プロジェクトルートは3階層上です
project_root = notebook_dir.parent.parent.parent

# プロジェクトルートをPythonの検索パスに追加
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
from groom import GroomAim2, \
    save_to_parquet, load_from_parquet, \
        save_model, load_model

In [3]:
self = GroomAim2()

## data準備

### raw_nose_face_dfの読み込み

In [4]:
raw_nose_face_df = load_from_parquet(
    "../../../data/03_aim_1_data/corrected_nose_face_df.parquet"
)

# 不要な列を削除
raw_nose_face_df = raw_nose_face_df.drop(
    columns=['t0_flag', 
             'delta_face','delta_nose',
             'shade', 'shade_condition', 
             'is_role_swapped', 'has_Sg',], 
    errors='ignore'
)

--- 復元完了 ---
復元されたDataFrameのshape: (80367, 21)


In [5]:
print(raw_nose_face_df.columns)

Index(['delta_time', 'sampling_id', 'datetime', 'name', 'id', 'sex',
       'behavior', 'to', 'from', 'kin', 'corrected_delta_nose',
       'corrected_delta_face', 'nose-face', 'corrected_nose-face'],
      dtype='object')


In [6]:
# 'from'列と'to'列の両方に値がある（NaNでない）行を抽出
filtered_df = raw_nose_face_df.dropna(subset=['from', 'to'])

# 列を指定して表示
print(filtered_df[['sampling_id', 'name', 'from', 'to']])

Empty DataFrame
Columns: [sampling_id, name, from, to]
Index: []


オスは相手方としてしか出現しない

In [7]:
print(len(raw_nose_face_df[raw_nose_face_df['name'] == 'Kobu']))
print(len(raw_nose_face_df[raw_nose_face_df['name'] == 'Nishin']))
print(len(raw_nose_face_df[raw_nose_face_df['name'] == 'Gure']))

0
0
0


### grooming_dfとgroomed_dfの作成

#### まずは分割

In [8]:
raw_grooming_df = raw_nose_face_df[raw_nose_face_df['behavior']=='grooming'].copy()
raw_groomed_df = raw_nose_face_df[raw_nose_face_df['behavior']=='groomed'].copy()

### fromおよびto列の補完

In [9]:
grooming_df = self.fix_grooming_records(
    raw_grooming_df,
    'grooming'
)

--- Mode: grooming (Counterpart column: to) ---
記録ミスの可能性がある sampling_id: [8, 73]
入れ替え処理が完了しました。
from 列を name 列の値で補完しました。

[処理結果の確認(代表各1行)]
      sampling_id      name      from        to
602             3     Botan     Botan   Komatsu
1204            5     Botan     Botan   Komatsu
1806            8     Botan     Botan   Komatsu
2408           10     Botan     Botan  Botaninf
3311           15     Kanna     Kanna      Yuna
4214           20  Tsutsuji  Tsutsuji      Gure
4515           22  Tsutsuji  Tsutsuji      Kobu
4816           24  Tsutsuji  Tsutsuji      Kobu
5117           26  Tsutsuji  Tsutsuji     Tsuwa
5719           28  Tsutsuji  Tsutsuji     Tsuwa


In [10]:
groomed_df = self.fix_grooming_records(
    raw_groomed_df,
    'groomed'
)

--- Mode: groomed (Counterpart column: from) ---
記録ミスの可能性がある sampling_id: [7, 51, 72]
入れ替え処理が完了しました。
to 列を name 列の値で補完しました。

[処理結果の確認(代表各1行)]
       sampling_id      name     from        to
903              4     Botan  Komatsu     Botan
1505             7     Botan  Komatsu     Botan
3612            17     Kanna     Kobu     Kanna
5418            27  Tsutsuji    Tsuwa  Tsutsuji
7224            33     Tsuwa  Tsukasa     Tsuwa
7826            35     Tsuwa  Tsukasa     Tsuwa
9632            42    Takana     Tabu    Takana
10234           44    Binega     Hiba    Binega
11739           51   Komatsu   Namiru   Komatsu
12642           56      Yuna    Kanna      Yuna


## rankの考慮

### rank_dfの読み込み

In [11]:
rank_df = load_from_parquet(
    "../../../data/01_relationship_data/rank_df.parquet"
)

--- 復元完了 ---
復元されたDataFrameのshape: (22, 2)


### オスを含めたrank_dictの作成

In [12]:
# 1. 名前から順位を引くための辞書を作成
# rank_df にないオスがいれば、非常に高い順位（例：0）を割り当てる
rank_dict = rank_df.set_index('name')['rank'].to_dict()

# オスのリスト（個別に順位0を割り当て）
males = ['Kobu', 'Nishin', 'Gure']
for male in males:
    rank_dict[male] = 0

毛繕いの順位方向を`rank_direction`列に追加

In [13]:
grooming_df_rank = self.add_rank_direction(
    grooming_df,
    rank_dict
)

# 順位が同じペア（'equal'）はほぼなかったので捨てても良い
drop_cols = ['unknown']  
grooming_df_rank = \
    grooming_df_rank[~grooming_df_rank['rank_direction'].isin(drop_cols)]

順位方向の判定が完了しました。
rank_direction
low_to_high    19565
high_to_low    16014
equal            240
Name: count, dtype: int64


In [14]:
groomed_df_rank = self.add_rank_direction(
    groomed_df,
    rank_dict
)

# 順位が同じペア（'equal'）はほぼなかったので捨てても良い
drop_cols = ['unknown']  
groomed_df_rank = \
    groomed_df_rank[~groomed_df_rank['rank_direction'].isin(drop_cols)]

順位方向の判定が完了しました。
rank_direction
high_to_low    15953
low_to_high    15652
Name: count, dtype: int64


In [15]:
print(grooming_df_rank[['sampling_id', 'name', 'from', 'to', 'rank_direction']].sample(10))

       sampling_id     name     from       to rank_direction
46686          186     Yone     Yone    Peach    high_to_low
54226          218     Kizu     Kizu   Hirugi    high_to_low
23372           97  Yotsuba  Yotsuba   hamayu    high_to_low
74517          306  Yotsuba  Yotsuba   Takana    low_to_high
41301          165    Mikan    Mikan  Yotsuba    low_to_high
68608          277    Botan    Botan     Kobu    low_to_high
45714          181   Binega   Binega  Yotsuba    low_to_high
23461           97  Yotsuba  Yotsuba   hamayu    high_to_low
39624          159    Mikan    Mikan     Beni    low_to_high
3479            15    Kanna    Kanna     Yuna    low_to_high


## tie_strength

In [16]:
grooming_df_tie = grooming_df_rank.copy()
groomed_df_tie = groomed_df_rank.copy()

## centrality

### centralityのdictを作成

In [17]:
centrality_df = load_from_parquet(
    "../../../data/01_relationship_data/centrality_df.parquet"
)

--- 復元完了 ---
復元されたDataFrameのshape: (25, 2)


In [18]:
# 名前から中心性を引くための辞書を作成
centrality_dict = centrality_df.set_index('name')['centrality'].to_dict()
print(centrality_dict)

{'Kannna': 22.0, 'Komatsu': 21.0, 'Botan': 17.0, 'Beni': 15.0, 'Yotsuba': 14.0, 'Tsutsuji': 12.0, 'Hamayu': 12.0, 'Kobu': 12.0, 'Tane': 11.0, 'Mikan': 11.0, 'Tsukasa': 10.0, 'Yuna': 10.0, 'Kizu': 9.0, 'Hinoki': 7.0, 'Yone': 7.0, 'Binega': 6.0, 'Gure': 6.0, 'Tabu': 4.0, 'Takana': 3.0, 'Hiba': 3.0, 'Hado': 3.0, 'Tsuwa': 3.0, 'Nishin': 2.0, 'Peach': 2.0, 'Teriba': 2.0}


### 中心性の方向性を付与（`centrality_direction`列）

In [19]:
grooming_df_centrality = self.add_centrality_direction(
    grooming_df_tie,
    centrality_dict
)

中心性方向の判定が完了しました。
centrality_direction
high_to_low    24983
low_to_high     9331
equal           1505
Name: count, dtype: int64


In [20]:
groomed_df_centrality = self.add_centrality_direction(
    groomed_df_tie,
    centrality_dict
)

中心性方向の判定が完了しました。
centrality_direction
low_to_high    19866
high_to_low    10836
equal            903
Name: count, dtype: int64


## 最終的なdfの保存

In [21]:
save_to_parquet(
    grooming_df_centrality,
    "../../../data/04_aim_2_data/01_add_label/labeled_grooming_df.parquet"
)

--- 保存完了 ---
'../../../data/04_aim_2_data/01_add_label/labeled_grooming_df.parquet' として保存されました。ファイルサイズ: 1336.00 KB


In [22]:
save_to_parquet(
    groomed_df_centrality,
    "../../../data/04_aim_2_data/01_add_label/labeled_groomed_df.parquet"
)

--- 保存完了 ---
'../../../data/04_aim_2_data/01_add_label/labeled_groomed_df.parquet' として保存されました。ファイルサイズ: 1132.94 KB
